In [23]:
#CONSTANTS:
c_train_df_raw_file='train_v1.csv'
c_valid_df_raw_file='valid_v1.pkl'
c_valid_df_prep_file='valid_prep_v1.pkl'
c_train_df_prep_file='train_df_prep_file.pkl'
c_train_df_prep_full_file='train_df_prep_full_file.pkl'
c_train_df_raw_full_file='train_df_raw_full_file.pkl'
c_test_df_prep='test_df_prep.pkl'
path = 'input/'
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [11]:
import pandas as pd
import gc
gc.collect()

710

In [12]:
print('data prep...')
most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
least_freq_hours_in_test_data = [6, 11, 15]

def prep_data( df ):
    df['hour'] = pd.to_datetime(df.click_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(df.click_time).dt.day.astype('uint8')
    df.drop(['click_time'], axis=1, inplace=True)
    gc.collect()
    
    df['in_test_hh'] = (   3 
                         - 2*df['hour'].isin(  most_freq_hours_in_test_data ) 
                         - 1*df['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')
    print( df.info() )
    
    
    print('group_by:','ip,day,hourcategrou in test')
    gp=df[['ip','day','in_test_hh','channel']].groupby(by=['ip','day','in_test_hh'])[['channel']].count().reset_index().rename(index=str,columns={'channel':'nip_day_test_hh'})
    df=df.merge(gp,on=['ip','day','in_test_hh'],how='left')
#     print('group by : ip_day_test_hh')
#     gp = df[['ip', 'day', 'in_test_hh', 'channel']].groupby(by=['ip', 'day',
#              'in_test_hh'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_day_test_hh'})
#     df = df.merge(gp, on=['ip','day','in_test_hh'], how='left')
    del gp
    df.drop(['in_test_hh'], axis=1, inplace=True)
    print( "nip_day_test_hh max value = ", df.nip_day_test_hh.max() )
    df['nip_day_test_hh'] = df['nip_day_test_hh'].astype('uint32')
    gc.collect()
    print( df.info() )

    print('group by : ip_day_hh')
    gp = df[['ip', 'day', 'hour', 'channel']].groupby(by=['ip', 'day', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_day_hh'})
    df = df.merge(gp, on=['ip','day','hour'], how='left')
    del gp
    print( "nip_day_hh max value = ", df.nip_day_hh.max() )
    df['nip_day_hh'] = df['nip_day_hh'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_os')
    gp = df[['ip', 'os', 'hour', 'channel']].groupby(by=['ip', 'os', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_os'})
    df = df.merge(gp, on=['ip','os','hour'], how='left')
    del gp
    print( "nip_hh_os max value = ", df.nip_hh_os.max() )
    df['nip_hh_os'] = df['nip_hh_os'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_app')
    gp = df[['ip', 'app', 'hour', 'channel']].groupby(by=['ip', 'app', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_app'})
    df = df.merge(gp, on=['ip','app','hour'], how='left')
    del gp
    print( "nip_hh_app max value = ", df.nip_hh_app.max() )
    df['nip_hh_app'] = df['nip_hh_app'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_dev')
    gp = df[['ip', 'device', 'hour', 'channel']].groupby(by=['ip', 'device', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_dev'})
    df = df.merge(gp, on=['ip','device','hour'], how='left')
    del gp
    print( "nip_hh_dev max value = ", df.nip_hh_dev.max() )
    df['nip_hh_dev'] = df['nip_hh_dev'].astype('uint32')
    gc.collect()
    print( df.info() )

    df.drop( ['ip','day'], axis=1, inplace=True )
    gc.collect()
    print( df.info() )
    
    return( df )b

data prep...


In [24]:
valid_df_raw= pd.read_pickle(c_valid_df_raw_file)

In [26]:
valid_df_raw.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_date
144709140,92588,2,1,13,212,2017-11-09 04:00:01,NaT,0,2017-11-09
144709141,106347,18,1,18,107,2017-11-09 04:00:01,NaT,0,2017-11-09
144709142,62440,18,1,17,107,2017-11-09 04:00:01,NaT,0,2017-11-09
144709143,27597,18,1,25,121,2017-11-09 04:00:01,NaT,0,2017-11-09
144709144,161604,2,1,15,236,2017-11-09 04:00:01,NaT,0,2017-11-09


In [27]:
valid_df_prep = prep_data(valid_df_raw)
del valid_df_raw
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37168192 entries, 144709140 to 181877331
Data columns (total 11 columns):
ip                 int64
app                int64
device             int64
os                 int64
channel            int64
attributed_time    datetime64[ns]
is_attributed      int64
click_date         object
hour               uint8
day                uint8
in_test_hh         uint8
dtypes: datetime64[ns](1), int64(6), object(1), uint8(3)
memory usage: 2.6+ GB
None
group_by: ip,day,hourcategrou in test
nip_day_test_hh max value =  174371
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37168192 entries, 0 to 37168191
Data columns (total 11 columns):
ip                 int64
app                int64
device             int64
os                 int64
channel            int64
attributed_time    datetime64[ns]
is_attributed      int64
click_date         object
hour               uint8
day                uint8
nip_day_test_hh    uint32
dtypes: datetime64[ns](1), int64

152

In [29]:
valid_df_prep.to_pickle(c_valid_df_prep_file)

In [13]:
all_data=False

def prep_train_data(raw,prep):
    import os
#     os.remove(prep)
#     print('deleted ',prep)
    print( "Train info before: ")
    train_df_raw= pd.read_csv(raw)
    print( train_df_raw.info() )
    train_df_prep = prep_data(train_df_raw)
    del train_df_raw
    gc.collect()
    print( "Train info after: ")
    print( train_df_prep.info() )
    train_df_prep.to_pickle(prep)
    del train_df_prep
    gc.collect()

train_raw=c_train_df_raw_file
train_prep=c_train_df_prep_file
    
if all_data:
    train_raw=c_train_df_raw_full_file
    train_prep=c_train_df_prep_full_file
    
prep_train_data(train_raw,train_prep)

Train info before: 


ModuleNotFoundError: No module named 'p,app,device,os,channel,click_time,attributed_time,is_attributed,click_date'

In [ ]:
print('hi')

In [ ]:
print('load test...')
test_cols = ['ip','app','device','os', 'channel', 'click_time', 'click_id']
test_df = pd.read_csv(path+"test.csv", dtype=dtypes, usecols=test_cols)
test_df = prep_data( test_df )
test_df.to_pickle(c_test_df_prep)
gc.collect()


In [13]:
print('hi1')

hi1


In [8]:
# c_train_df_raw_full_file='train_df_raw_full_file.pkl'

In [12]:
# # def prep_train_full_data():
# print( "Train info before: ")
# train_df_raw= pd.read_pickle(c_train_df_raw_full_file)
# print( train_df_raw.info() )
# train_df_prep = prep_data(train_df_raw)
# # prep_train_full_data()

Train info before: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       object
is_attributed    uint8
dtypes: object(1), uint16(4), uint32(1), uint8(1)
memory usage: 3.6+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 9 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
is_attributed    uint8
hour             uint8
day              uint8
in_test_hh       uint8
dtypes: uint16(4), uint32(1), uint8(4)
memory usage: 2.8 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
app              uint16
device           uint16
os               uint16
channel          uint16
is_attributed  

In [13]:
# # del train_df_raw
# # gc.collect()
# print( "Train info after: ")
# print( train_df_prep.info() )
# train_df_prep.to_pickle(c_train_df_prep_full_file)
# del train_df_prep
# gc.collect()


Train info after: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
app              uint16
device           uint16
os               uint16
channel          uint16
is_attributed    uint8
hour             uint8
in_test_hh       uint8
dtypes: uint16(4), uint8(3)
memory usage: 1.9 GB
None


34